In [1]:
import os
ionq_token=os.getenv('IONQ_API_TOKEN')
from qiskit_ionq import IonQProvider
from threading import Thread
provider = IonQProvider(ionq_token)

# show supported backends
print(provider.backends()) 

# create backends for simulator and hardware
backend_simq = provider.get_backend("ionq_simulator")
backend_qpu = provider.get_backend("ionq_qpu")

[<IonQSimulatorBackend('ionq_simulator')>, <IonQQPUBackend('ionq_qpu')>]


In [2]:
from qiskit_ionq import GPIGate, GPI2Gate, MSGate
backend_simq2 = provider.get_backend("ionq_simulator", gateset="native")
backend_qpu2 = provider.get_backend("ionq_qpu", gateset="native")

backend_simq = provider.get_backend("ionq_simulator")
backend_qpu = provider.get_backend("ionq_qpu")

In [3]:
#Native quantum circuit using GPI, GPI2 gates to generate 1 byte

from qiskit import QuantumCircuit

nbyte = QuantumCircuit(8)

for i in range(8):
    nbyte.append(GPI2Gate(0.25), [i])
    nbyte.append(GPIGate(0.5), [i])    

In [4]:
#Standard gate-set used to generate 1 byte using the non native gateset

non_nbyte = QuantumCircuit(8,8)
for i in range(8):
    non_nbyte.h(i)

In [5]:
#generates a byte, using 8 hadamards and single shot jobs (non-native gateset)
def gen_byte():
    qpu_job = backend_qpu.run(non_nbyte, shots = 1)
    counts = qpu_job.get_counts()
    byte = list(counts.keys())[0]
    return byte

In [6]:
from array import *

def gen_n_bytes(bytes, bin_array, file):
    bits = open(file, "ab")

    for i in range(bytes):
        byte = gen_byte()
        bin_array.append(int(byte,2))

    bin_array.tofile(bits)

In [7]:
#same method but now using the native gate set
def gen_nbyte():
    qpu_job = backend_qpu2.run(nbyte, shots = 1)
    counts = qpu_job.get_counts()
    byte = list(counts.keys())[0]
    return byte

In [8]:
#same method but now using the native gate set
from array import *
import os

def gen_n_nbytes(bytes, bin_array, file):
    bits = open(file, "ab")

    for i in range(bytes):
        byte = gen_nbyte()
        bin_array.append(int(byte,2))

    bin_array.tofile(bits)

In [10]:
import time

start = time.time()

#User will be prompted select how many bytes to generate.
print("Number of bytes to generate: ")
bytes = int(input())

my_job = [None] * bytes
#Creating a job list in order to reduce queue time
#Here we are queueing bytes single-shot jobs that each generate a byte

for i in range(bytes):
    my_job[i] = backend_qpu.run(non_nbyte, shots = 1)
    if(i % 100 == 0):
        time.sleep(100)
        print(i)

Number of bytes to generate: 
8
0


In [11]:
from qiskit.providers.jobstatus import JobStatus

my_results = []
for i in range(bytes):
    if my_job[i].status() is not JobStatus.DONE:
        time.sleep(100)
    else:
        my_results.append(list(my_job[i].get_counts().keys())[0])

In [12]:
time = time.time() - start

In [13]:
print(f"10000 bytes generated in {time} seconds.")
print("Generation rate (bps):", 10000*8/time)

10000 bytes generated in 931.6393399238586 seconds.
Generation rate (bps): 85.87013941096376


In [18]:
print(my_job[0].get_counts())

{'00000000': 1}


In [ ]:
print(my_job[99].get_counts())

In [ ]:
import time

print("Total time elapsed:", time.time() - start)